<a href="https://colab.research.google.com/github/angwelo/ASSIGNMENT-/blob/main/Retrieval_Augmented_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## INSTALLING LIBRARIES

In [1]:
%pip install langchain-groq langchain sentence-transformers langchain-community  pypdf chromadb

In [2]:
#importing libraries
from sentence_transformers import SentenceTransformer
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.storage import InMemoryStore
from langchain_groq import ChatGroq


In [3]:
doc_path="/content/alice_in_wonderland.pdf"
loader= PyPDFLoader(doc_path)
docs=loader.load()
print(len(docs))

84


In [4]:
#initializing parent and text splitters
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=100,
)

child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=50,
)


In [5]:
#initializing the embedding model
bge_model=SentenceTransformer("BAAI/bge-base-en-v1.5")
class BGEEmbeddings:
  def embed_documents(self, text):
    return bge_model.encode(text,batch_size=8,normalize_embeddings=True).tolist()
  def embed_query(self, query):
    return bge_model.encode([query],normalize_embeddings=True).tolist()[0]




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
#creating the vector store
wonderland_vector_store=Chroma(
    collection_name="wonderland",
    embedding_function=BGEEmbeddings(),
    persist_directory="./chroma",
)



/tmp/ipython-input-783672564.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  wonderland_vector_store=Chroma(


In [7]:
#creating in-memory store
store=InMemoryStore()

In [8]:
#creating retrievers
retriever=ParentDocumentRetriever(
    vectorstore=wonderland_vector_store,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [9]:
#adding documents into the vector store
retriever.add_documents(docs)

In [11]:
#creating a child retriever
child_retriever=wonderland_vector_store.as_retriever()

In [12]:
#initializing the LLM
llm=ChatGroq(
   groq_api_key=userdata.get('alice_in_wonderland'),
   model_name="llama-3.3-70b-versatile",
   temperature=0.7

)

In [13]:
#creating the template
template="""
 Answer the following questions according to the context given.
 If the question is out of context,just say the question is out of context Kindly give me another question.
 Do not try to answer a question out of context.
 {context}
 Question: {question}
"""


In [15]:
#creating the chain
qa_chain=PromptTemplate(template=template,input_variables=["context","question"])

In [16]:
chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":qa_chain},
    return_source_documents=True
)

In [21]:
question="How does the Queen of Hearts usually react when she is angry?"
answer=chain({"query":question})
print(answer["result"])

The Queen of Hearts usually reacts by shouting "Off with their head!" when she is angry. This is her typical response to any situation that displeases her, and she often calls for the execution of those who disagree with her or make mistakes.
